# Constitution du cloud MinIO Client 

Le fichier diffusion du cloud MinIO Client est composé :
- du fichier 'cds_data' qui regroupe les données issues du modèle de projection CMIP6 mis à disposition par le Climate Data Store de Copernicus 
- du fichier 'puffin_data' 

Ce notebook a pour objectif de centraliser les requêtes API réalisées pour obtenir les données climatiques issues du modèle prédictif CMIP6 et leur écriture sur le cloud ainsi que l'écriture des données relatives aux macaraux moines(Atlantic puffins en anglais).

## API - Climate Data Store

In [9]:
import cdsapi
import pandas as pd
import s3fs
import zipfile
import xarray as xr
import dotenv
import os


# 1. Définir les chemins dynamiques dans un sous-dossier "data/"
data_dir = "./data"  # Sous-dossier pour centraliser les données
os.makedirs(data_dir, exist_ok=True)  # Crée le dossier s'il n'existe pas
temp_dir = os.path.join(data_dir, "extracted_files") # Sous-dossier d'extraction des fichiers zip
os.makedirs(temp_dir, exist_ok=True)  # Crée le dossier d'extraction

#### Sécurisation des codes d'accès de l'API CDS dans un fichier .env

In [4]:
from dotenv import load_dotenv

# Charger les variables depuis le fichier .env
load_dotenv()

# Récupérer les variables d'environnement
cdsapi_url = os.getenv("cdsapi_url")
cdsapi_key = os.getenv("cdsapi_key")

# Construire le chemin vers le fichier .cdsapirc dans le répertoire utilisateur
home_dir = os.path.expanduser("~")
file_path = os.path.join(home_dir, ".cdsapirc")

# Écrire le fichier .cdsapirc avec les variables
with open(file_path, 'w') as file:
    file.write(f"""
    url: {cdsapi_url}
    key: {cdsapi_key}
""")
print(f"Le fichier .cdsapirc a été créé à : {file_path}")


Le fichier .cdsapirc a été créé à : /home/onyxia/.cdsapirc


### API CDS n°1 
Modèle : hadgem3_gc31_ll (UK)
Données : historiques [1950 - 2014] 
Localisation : îles Vestmann (Islande)
Variable : Sea Surface Temperature

#### Requête API 
Utilisation du module "cdsapi" développé par le Climate Data Store et écriture du fichier de téléchargement dans le sous-dossier 'data'

In [8]:
dataset = "projections-cmip6"
request = {
    "temporal_resolution": "monthly",
    "experiment": "historical",
    "variable": "sea_surface_temperature",
    "model": "hadgem3_gc31_ll",
    "year": [
        "1950", "1951", "1952",
        "1953", "1954", "1955",
        "1956", "1957", "1958",
        "1959", "1960", "1961",
        "1962", "1963", "1964",
        "1965", "1966", "1967",
        "1968", "1969", "1970",
        "1971", "1972", "1973",
        "1974", "1975", "1976",
        "1977", "1978", "1979",
        "1980", "1981", "1982",
        "1983", "1984", "1985",
        "1986", "1987", "1988",
        "1989", "1990", "1991",
        "1992", "1993", "1994",
        "1995", "1996", "1997",
        "1998", "1999", "2000",
        "2001", "2002", "2003",
        "2004", "2005", "2006",
        "2007", "2008", "2009",
        "2010", "2011", "2012",
        "2013", "2014"
    ],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "area": [63.5, 20.5, 63.3, 20.1],  # Zone : Nord, Ouest, Sud, Est
}


final_zip_path = os.path.join(data_dir, "cds_output_vestmann.zip")
client.retrieve(dataset, request).download(final_zip_path)




2024-12-17 10:33:45,244 INFO Request ID is e110e359-9be4-4df7-84a5-8e8bd9ac03ab
2024-12-17 10:33:45,810 INFO status has been updated to accepted
2024-12-17 10:33:51,672 INFO status has been updated to running
2024-12-17 10:33:55,113 INFO status has been updated to successful


'./data/cds_output_vestmann.zip'

#### Extraction du fichier ZIP

In [11]:
with zipfile.ZipFile(final_zip_path, 'r') as zip_ref:
    zip_ref.extractall(temp_dir)  # Extraction dans extracted_files
print(f"Fichiers extraits dans : {temp_dir}")

Fichiers extraits dans : ./data/extracted_files


#### Ecriture du fichier extrait sur le cloud MinIO Client 

In [13]:
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})
MY_BUCKET = "esam"
print(fs.ls(MY_BUCKET))
minio_path = f'{MY_BUCKET}/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_historical_r1i1p1f3_gn_19500116-20141216.nc'


with fs.open(minio_path, "wb") as minio_file:  # Fichier cible sur MinIO
    with open(file_path, "rb") as local_file:  # Fichier source local
        minio_file.write(local_file.read())

print(fs.ls(f"{MY_BUCKET}/diffusion/cds_data"))

['esam/diffusion']
['esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_historical_r1i1p1f3_gn_19500116-20141216.nc', 'esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_20150116-20490616.nc', 'esam/diffusion/cds_data/tos_Omon_MPI-ESM1-2-HR_historical_r1i1p1f1_gn_20000116-20140616.nc']


### API CDS n°2 - SSP 5-8.5 (scénario le plus pessimiste)
Modèle : hadgem3_gc31_ll (UK)
Données : prédictives [2015 - 2049] 
Localisation : îles Vestmann (Islande)
Variable : Sea Surface Temperature

#### Requête API 

In [14]:
dataset = "projections-cmip6"
request = {
    "temporal_resolution": "monthly",
    "experiment": "ssp5_8_5",
    "variable": "sea_surface_temperature",
    "model": "hadgem3_gc31_ll",
    "year": [
        "2015", "2016", "2017",
        "2018", "2019", "2020",
        "2021", "2022", "2023",
        "2024", "2025", "2026",
        "2027", "2028", "2029",
        "2030", "2031", "2032",
        "2033", "2034", "2035",
        "2036", "2037", "2038",
        "2039", "2040", "2041",
        "2042", "2043", "2044",
        "2045", "2046", "2047",
        "2048", "2049"
    ],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "area": [63.5, 20.5, 63.3, 20.1],  # Zone : Nord, Ouest, Sud, Est
}

final_zip_path = os.path.join(data_dir, "cds_output_vestmann_ssp5_8_5.zip")
client.retrieve(dataset, request).download(final_zip_path)


2024-12-17 10:43:21,307 INFO Request ID is 72a9940d-9c78-4476-b5ec-7dbff5d689b8
2024-12-17 10:43:21,457 INFO status has been updated to accepted
2024-12-17 10:43:37,216 INFO status has been updated to running
2024-12-17 10:43:45,781 INFO status has been updated to successful


'./data/cds_output_vestmann_ssp5_8_5.zip'

#### Extraction du fichier ZIP

In [15]:
with zipfile.ZipFile(final_zip_path, 'r') as zip_ref:
    zip_ref.extractall(temp_dir)  # Extraction dans extracted_files
print(f"Fichiers extraits dans : {temp_dir}")

Fichiers extraits dans : ./data/extracted_files


#### Ecriture du fichier extrait sur le cloud MinIO Client

In [16]:
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})
MY_BUCKET = "esam"
print(fs.ls(MY_BUCKET))
minio_path = f'{MY_BUCKET}/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_20150116-20491216.nc'


with fs.open(minio_path, "wb") as minio_file:  # Fichier cible sur MinIO
    with open(file_path, "rb") as local_file:  # Fichier source local
        minio_file.write(local_file.read())

print(fs.ls(f"{MY_BUCKET}/diffusion/cds_data"))

['esam/diffusion']
['esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_historical_r1i1p1f3_gn_19500116-20141216.nc', 'esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_20150116-20490616.nc', 'esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_20150116-20491216.nc', 'esam/diffusion/cds_data/tos_Omon_MPI-ESM1-2-HR_historical_r1i1p1f1_gn_20000116-20140616.nc']


### API CDS n°2 - SSP 1-2.6 (scénario le plus optimiste)
Modèle : hadgem3_gc31_ll (UK)
Données : prédictives [2015 - 2049] 
Localisation : îles Vestmann (Islande)
Variable : Sea Surface Temperature

#### Requête API

In [17]:
dataset = "projections-cmip6"
request = {
    "temporal_resolution": "monthly",
    "experiment": "ssp1_2_6",
    "variable": "sea_surface_temperature",
    "model": "hadgem3_gc31_ll",
    "year": [
        "2015", "2016", "2017",
        "2018", "2019", "2020",
        "2021", "2022", "2023",
        "2024", "2025", "2026",
        "2027", "2028", "2029",
        "2030", "2031", "2032",
        "2033", "2034", "2035",
        "2036", "2037", "2038",
        "2039", "2040", "2041",
        "2042", "2043", "2044",
        "2045", "2046", "2047",
        "2048", "2049"
    ],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "area": [63.5, 20.5, 63.3, 20.1],  # Zone : Nord, Ouest, Sud, Est
}

final_zip_path = os.path.join(data_dir, "cds_output_vestmann_ssp1_2_6.zip")
client.retrieve(dataset, request).download(final_zip_path)


2024-12-17 11:09:46,003 INFO Request ID is dfc1936d-c2dc-48a1-9726-72ca27843d2c
2024-12-17 11:09:47,136 INFO status has been updated to accepted
2024-12-17 11:09:53,236 INFO status has been updated to running
2024-12-17 11:10:01,845 INFO status has been updated to successful


'./data/cds_output_vestmann_ssp1_2_6.zip'

#### Extraction du fichier ZIP

In [18]:
with zipfile.ZipFile(final_zip_path, 'r') as zip_ref:
    zip_ref.extractall(temp_dir)  # Extraction dans extracted_files
print(f"Fichiers extraits dans : {temp_dir}")

Fichiers extraits dans : ./data/extracted_files


#### Ecriture du fichier extrait sur le cloud MinIO Client

In [19]:
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})
MY_BUCKET = "esam"
print(fs.ls(MY_BUCKET))
minio_path = f'{MY_BUCKET}/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_ssp126_r1i1p1f3_gn_20150116-20491216.nc'


with fs.open(minio_path, "wb") as minio_file:  # Fichier cible sur MinIO
    with open(file_path, "rb") as local_file:  # Fichier source local
        minio_file.write(local_file.read())

print(fs.ls(f"{MY_BUCKET}/diffusion/cds_data"))

['esam/diffusion']
['esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_historical_r1i1p1f3_gn_19500116-20141216.nc', 'esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_ssp126_r1i1p1f3_gn_20150116-20491216.nc', 'esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_20150116-20491216.nc']


### API CDS n°4 - SSP 2-4.5 (scénario intermédiaire)
Modèle : hadgem3_gc31_ll (UK)
Données : prédictives [2015 - 2049] 
Localisation : îles Vestmann (Islande)
Variable : Sea Surface Temperature

#### Requête API

In [20]:
dataset = "projections-cmip6"
request = {
    "temporal_resolution": "monthly",
    "experiment": "ssp2_4_5",
    "variable": "sea_surface_temperature",
    "model": "hadgem3_gc31_ll",
    "year": [
        "2015", "2016", "2017",
        "2018", "2019", "2020",
        "2021", "2022", "2023",
        "2024", "2025", "2026",
        "2027", "2028", "2029",
        "2030", "2031", "2032",
        "2033", "2034", "2035",
        "2036", "2037", "2038",
        "2039", "2040", "2041",
        "2042", "2043", "2044",
        "2045", "2046", "2047",
        "2048", "2049"
    ],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "area": [63.5, 20.5, 63.3, 20.1],  # Zone : Nord, Ouest, Sud, Est
}

final_zip_path = os.path.join(data_dir, "cds_output_vestmann_ssp2_4_5.zip")
client.retrieve(dataset, request).download(final_zip_path)

2024-12-17 11:13:06,806 INFO Request ID is 772c0b77-2eb8-4449-a197-29b53e86e197
2024-12-17 11:13:06,897 INFO status has been updated to accepted
2024-12-17 11:13:15,648 INFO status has been updated to running
2024-12-17 11:13:28,495 INFO status has been updated to successful


'./data/cds_output_vestmann_ssp2_4_5.zip'

#### Extraction du fichier ZIP

In [21]:
with zipfile.ZipFile(final_zip_path, 'r') as zip_ref:
    zip_ref.extractall(temp_dir)  # Extraction dans extracted_files
print(f"Fichiers extraits dans : {temp_dir}")

Fichiers extraits dans : ./data/extracted_files


#### Ecriture du fichier extrait sur le cloud MinIO Client

In [22]:
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})
MY_BUCKET = "esam"
print(fs.ls(MY_BUCKET))
minio_path = f'{MY_BUCKET}/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_ssp245_r1i1p1f3_gn_20150116-20491216.nc'


with fs.open(minio_path, "wb") as minio_file:  # Fichier cible sur MinIO
    with open(file_path, "rb") as local_file:  # Fichier source local
        minio_file.write(local_file.read())

print(fs.ls(f"{MY_BUCKET}/diffusion/cds_data"))

['esam/diffusion']
['esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_historical_r1i1p1f3_gn_19500116-20141216.nc', 'esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_ssp126_r1i1p1f3_gn_20150116-20491216.nc', 'esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_ssp245_r1i1p1f3_gn_20150116-20491216.nc', 'esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_20150116-20491216.nc']
